# Football Players Network Analysis and Visualization

Jonas Zhonghan Xie  
jonasxie@umich.edu  
2024/11/30

## Prepare the data

This project uses data from the [European Soccer Database](https://www.kaggle.com/hugomathien/soccer) on Kaggle. The database contains data for soccer matches, players, and teams from several European countries from 2008 to 2016. The database is stored in a SQLite database, which can be accessed using the `sqlite3` library in Python.

In [173]:
# import kagglehub

# kagglepath = kagglehub.dataset_download("davidcariboo/player-scores")
# print(kagglepath)

In [104]:
import pandas as pd
import numpy as np
import networkx as nx
import gc
import math

In [3]:
clubs = pd.read_csv("./data/clubs.csv")

In [6]:
competitions = ['IT1', 'GB1', 'FR1', 'ES1', 'L1']
clubs = clubs[clubs['domestic_competition_id'].isin(competitions)]
clubs = clubs[['club_id', 'domestic_competition_id']]

In [14]:
players = pd.read_csv("./data/players.csv")

In [15]:
players.head(5)

,player_id,first_name,last_name,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,foot,height_in_cm,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
0,10,Miroslav,Klose,Miroslav Klose,2015,398,miroslav-klose,Poland,Opole,Germany,...,right,184.0,NaN,ASBW Sport Marketing,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,IT1,Società Sportiva Lazio S.p.A.,1000000.0,30000000.0
1,26,Roman,Weidenfeller,Roman Weidenfeller,2017,16,roman-weidenfeller,Germany,Diez,Germany,...,left,190.0,NaN,Neubauer 13 GmbH,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/roman-weidenfe...,L1,Borussia Dortmund,750000.0,8000000.0
2,65,Dimitar,Berbatov,Dimitar Berbatov,2015,1091,dimitar-berbatov,Bulgaria,Blagoevgrad,Bulgaria,...,NaN,NaN,NaN,CSKA-AS-23 Ltd.,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/dimitar-berbat...,GR1,Panthessalonikios Athlitikos Omilos Konstantin...,1000000.0,34500000.0
3,77,NaN,Lúcio,Lúcio,2012,506,lucio,Brazil,Brasília,Brazil,...,NaN,NaN,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/lucio/profil/s...,IT1,Juventus Football Club,200000.0,24500000.0
4,80,Tom,Starke,Tom Starke,2017,27,tom-starke,East Germany (GDR),Freital,Germany,...,right,194.0,NaN,IFM,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/tom-starke/pro...,L1,FC Bayern München,100000.0,3000000.0


In [28]:
df_players = players[
    [
        "player_id",
        "first_name",
        "last_name",
        "name",
        "last_season",
        "current_club_id",
        "current_club_name",
        "position",
        "country_of_citizenship",
        "highest_market_value_in_eur",
    ]
]
players.columns

Index(['player_id', 'first_name', 'last_name', 'name', 'last_season',
       'current_club_id', 'player_code', 'country_of_birth', 'city_of_birth',
       'country_of_citizenship', 'date_of_birth', 'sub_position', 'position',
       'foot', 'height_in_cm', 'contract_expiration_date', 'agent_name',
       'image_url', 'url', 'current_club_domestic_competition_id',
       'current_club_name', 'market_value_in_eur',
       'highest_market_value_in_eur'],
      dtype='object')

In [29]:
players_clubs = pd.merge(df_players, clubs, left_on="current_club_id", right_on="club_id")

In [30]:
players_clubs.head(5)

,player_id,first_name,last_name,name,last_season,current_club_id,current_club_name,position,country_of_citizenship,highest_market_value_in_eur,club_id,domestic_competition_id
0,10,Miroslav,Klose,Miroslav Klose,2015,398,Società Sportiva Lazio S.p.A.,Attack,Germany,30000000.0,398,IT1
1,26,Roman,Weidenfeller,Roman Weidenfeller,2017,16,Borussia Dortmund,Goalkeeper,Germany,8000000.0,16,L1
2,77,NaN,Lúcio,Lúcio,2012,506,Juventus Football Club,Defender,Brazil,24500000.0,506,IT1
3,80,Tom,Starke,Tom Starke,2017,27,FC Bayern München,Goalkeeper,Germany,3000000.0,27,L1
4,123,Christoph,Metzelder,Christoph Metzelder,2012,33,FC Schalke 04,Defender,Germany,9500000.0,33,L1


In [31]:
players_clubs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11665 entries, 0 to 11664
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   player_id                    11665 non-null  int64  
 1   first_name                   11220 non-null  object 
 2   last_name                    11665 non-null  object 
 3   name                         11665 non-null  object 
 4   last_season                  11665 non-null  int64  
 5   current_club_id              11665 non-null  int64  
 6   current_club_name            11665 non-null  object 
 7   position                     11665 non-null  object 
 8   country_of_citizenship       11540 non-null  object 
 9   highest_market_value_in_eur  11263 non-null  float64
 10  club_id                      11665 non-null  int64  
 11  domestic_competition_id      11665 non-null  object 
dtypes: float64(1), int64(4), object(7)
memory usage: 1.1+ MB


In [32]:
# Identify the teammates and the players from the same citizenship
# Transform the dataset to like "player1, player2, Same Country"
# This will be used to create the network graph
# The graph will have the players as nodes and the connections will be the teammates and the players from the same country

# Create a new dataframe with the columns player_id, player_code, country_of_citizenship
df_players_country = players_clubs[["player_id", "name", "country_of_citizenship"]]

# Merge the dataframe with itself to get the teammates and the players from the same country
df_players_country = df_players_country.merge(df_players_country, on="country_of_citizenship")
df_players_country.head(5)

# Drop the same person
df_players_country = df_players_country[
    df_players_country["name_x"] != df_players_country["name_y"]
]
df_players_country["Relationship"] = "Same Country"
df_players_country.columns = [
    "player1",
    "player1_name",
    "tie",
    "player2",
    "player2_name",
    "relation",
]

# rule out the same pair
df_players_country = df_players_country[df_players_country["player1"] < df_players_country["player2"]]

In [33]:
df_players_country.head(5)

,player1,player1_name,tie,player2,player2_name,relation
1,10,Miroslav Klose,Germany,26,Roman Weidenfeller,Same Country
2,10,Miroslav Klose,Germany,80,Tom Starke,Same Country
3,10,Miroslav Klose,Germany,123,Christoph Metzelder,Same Country
4,10,Miroslav Klose,Germany,162,Marc Ziegler,Same Country
5,10,Miroslav Klose,Germany,258,Gerald Asamoah,Same Country


In [34]:
df_players_country.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5280062 entries, 1 to 10570328
Data columns (total 6 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   player1       int64 
 1   player1_name  object
 2   tie           object
 3   player2       int64 
 4   player2_name  object
 5   relation      object
dtypes: int64(2), object(4)
memory usage: 282.0+ MB


In [35]:
# Identify the teammates
df_players_club = players_clubs[["player_id", "name", "current_club_id", "current_club_name"]]
df_players_club = df_players_club.merge(df_players_club, on="current_club_id")
df_players_club = df_players_club[df_players_club["name_x"] != df_players_club["name_y"]]
df_players_club.drop(columns=["current_club_id", "current_club_name_y"], inplace=True)
df_players_club["Relationship"] = "Club Teammate"
df_players_club.columns = ["player1", "player1_name", "tie", "player2", "player2_name", "relation"]

In [36]:
df_players_club.head(5)

,player1,player1_name,tie,player2,player2_name,relation
1,10,Miroslav Klose,Società Sportiva Lazio S.p.A.,4015,Louis Saha,Club Teammate
2,10,Miroslav Klose,Società Sportiva Lazio S.p.A.,5812,Cristian Brocchi,Club Teammate
3,10,Miroslav Klose,Società Sportiva Lazio S.p.A.,6026,Tommaso Rocchi,Club Teammate
4,10,Miroslav Klose,Società Sportiva Lazio S.p.A.,6043,Stefano Mauri,Club Teammate
5,10,Miroslav Klose,Società Sportiva Lazio S.p.A.,12518,Silvio Proto,Club Teammate


In [37]:
# Append two dataframes
df_players_relationship = pd.concat([df_players_country, df_players_club])
df_players_relationship.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6233898 entries, 1 to 965503
Data columns (total 6 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   player1       int64 
 1   player1_name  object
 2   tie           object
 3   player2       int64 
 4   player2_name  object
 5   relation      object
dtypes: int64(2), object(4)
memory usage: 332.9+ MB


In [38]:
# Save memory
del df_players_country, df_players_club

In [39]:
# Merge the original dataframe to take in the market value
df_players_relationship = pd.merge(
    df_players_relationship,
    players[["player_id", "highest_market_value_in_eur"]],
    left_on="player1",
    right_on="player_id",
    how="left",
)
df_players_relationship.rename(
    columns={"highest_market_value_in_eur": "player1_value"}, inplace=True
)

df_players_relationship = pd.merge(
    df_players_relationship,
    players[["player_id", "highest_market_value_in_eur"]],
    left_on="player2",
    right_on="player_id",
    how="left",
)
df_players_relationship.rename(
    columns={"highest_market_value_in_eur": "player2_value"}, inplace=True
)

# drop the columns with 0 or none market values
df_players_relationship = df_players_relationship[
    (df_players_relationship["player1_value"] > 0)
    & (df_players_relationship["player2_value"] > 0)
]

In [41]:
df_players_relationship.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5769644 entries, 0 to 6233896
Data columns (total 10 columns):
 #   Column         Dtype  
---  ------         -----  
 0   player1        int64  
 1   player1_name   object 
 2   tie            object 
 3   player2        int64  
 4   player2_name   object 
 5   relation       object 
 6   player_id_x    int64  
 7   player1_value  float64
 8   player_id_y    int64  
 9   player2_value  float64
dtypes: float64(2), int64(4), object(4)
memory usage: 484.2+ MB


In [42]:
del players  # Save memory

In [43]:
gc.collect()

526

## Network Analysis with NetworkX

In [44]:
from pyecharts.charts import Graph
from pyecharts import options as opts

In [125]:
# Define a new function to build up the network for a single player without using my previous code
# The size of nodes depends on the market value of the player
# The weight of the edges depends on the market value of the other player
def build_network(player, relation="all", limit=-1):
    def _find_neighbors(G, player1, player2, limit = -1):
        """
        Build up the path for the two players, with a limit of the number of connecting nodes.
        If the connecting nodes are more than the limit, return the nodes of top highest market value.
        If there is no path between the two players, return the original graph.

        Parameters
        ----------
        G : nx.Graph
            The original graph
        player1 : str
            The name of the first player
        player2 : str
            The name of the second player
        limit : int
            The limit of the number of connecting nodes
        """
        if nx.has_path(G, player1, player2):
            connected_nodes = nx.node_connected_component(G, player1)
            if player2 in connected_nodes:
                subG = G.subgraph(connected_nodes).copy()
                neighbors1 = set(G.neighbors(player1))
                neighbors2 = set(G.neighbors(player2))
                overlapping = neighbors1.intersection(neighbors2)
                if limit > 0 and len(overlapping) > limit:
                    sorted_neighbors = sorted(
                        overlapping,
                        key=lambda x: G.nodes[x].get("player_value", 0),
                        reverse=True,
                    )
                    overlapping = set(sorted_neighbors[:limit])
                subnodes = {player1, player2} | overlapping
                subG = G.subgraph(subnodes).copy()
                return subG, overlapping
        return G, set()

    if type(player) == str:
        player = [player]
    # Filter the dataframe for the player
    df_player = df_players_relationship[
        (df_players_relationship["player1_name"].isin(player))
        | (df_players_relationship["player2_name"].isin(player))
    ]
    if relation != "all":
        df_player = df_player[df_player["relation"] == relation]

    G = nx.Graph()

    if len(player)==2:
        player1, player2 = player
        full_graph = nx.Graph()
        for player1_name, player2_name, rel, player1_val, player2_val in zip(
            df_player["player1_name"],
            df_player["player2_name"],
            df_player["relation"],
            df_player["player1_value"],
            df_player["player2_value"],
        ):
            full_graph.add_node(player1_name, player_value=player1_val)
            full_graph.add_node(player2_name, player_value=player2_val)
            full_graph.add_edge(player1_name, player2_name, relation=rel)

        subG, overlapping = _find_neighbors(full_graph, player1, player2, limit)
        G.add_nodes_from(subG.nodes(data=True))
        G.add_edges_from(subG.edges(data=True))

        # Add or update attributes for overlapping nodes
        for node in G.nodes:
            G.nodes[node]["player_value"] = subG.nodes[node].get("player_value", 0)
            G.nodes[node]["overlap"] = node in overlapping
    else:
        # Build the graph for a single player
        for player1_name, player2_name, rel, player1_val, player2_val in zip(
            df_player["player1_name"],
            df_player["player2_name"],
            df_player["relation"],
            df_player["player1_value"],
            df_player["player2_value"],
        ):
            G.add_node(player1_name, player_value=player1_val)
            G.add_node(player2_name, player_value=player2_val)
            G.add_edge(player1_name, player2_name, relation=rel, weight=player2_val)

    return G

In [157]:
def illustrate_graph(G, players, title="Player Network"):
    """
    Visualize a NetworkX graph using Pyecharts with selected players on opposite sides.

    Parameters:
    - G (networkx.Graph): The NetworkX graph to visualize.
    - selected_players (list): List of two players to position on opposite sides.
    - title (str): Title of the graph visualization.

    Returns:
    - Generates an interactive HTML file to visualize the graph.
    """

    player1, player2 = players

    # Extract nodes and edges
    nodes = []
    edges = []

    edge_color_map = {"Club Teammate": "green", "Same Country": "blue"}
    legend_entries = [{"name": "Club Teammate", "color": "green"}, {"name": "Same Country", "color": "blue"}]

    for u, v, data in G.edges(data=True):
        edge_entry = {
            'source': u,
            'target': v,
            'value': data.get('weight', 1),
            'tooltip': data.get('relation'),
            'lineStyle': {'color': edge_color_map[data.get('relation')]}
        }
        edges.append(edge_entry)
    
    # Manually position the selected players
    manual_positions = {
        player1: {"x": -300, "y": 0},  # Far-left
        player2: {"x": 300, "y": 0},   # Far-right
    }

    middle_nodes = [node for node in G.nodes if node not in players]

    # Spread connecting nodes across a grid in the middle
    # Circular layout for middle nodes
    middle_nodes = [node for node in G.nodes if node not in players]
    radius = 100  # Radius of the circle
    angle_step = 2 * math.pi / len(middle_nodes) if middle_nodes else 0

    for i, node in enumerate(middle_nodes):
        angle = i * angle_step
        x_pos = radius * math.cos(angle)  # Compute x using cosine
        # prevent the x_pos from being 0
        y_pos = radius * math.sin(angle)  # Compute y using sine
        if abs(y_pos) < 10:
            y_pos += 10 if y_pos>=0 else -10
        manual_positions[node] = {"x": x_pos, "y": y_pos}

    max_value = max((data.get("player_value", 0) for _, data in G.nodes(data=True)), default=1)
    min_value = min((data.get("player_value", 0) for _, data in G.nodes(data=True)), default=0)

    for node, data in G.nodes(data=True):
        # Determine node size
        raw_value = data.get("player_value", 0)
        scaled_size = (
            10 + (40 * (raw_value - min_value) / (max_value - min_value)) if max_value > min_value else 20
        )

        # Assign manual position
        pos = manual_positions.get(node, {"x": 0, "y": 0})  # Default position at center if not specified
        color = "red" if data.get("overlap") else "blue"  # Overlapping nodes are red
        node_entry = {
            "name": node,
            "value": raw_value,
            "symbolSize": scaled_size,
            "itemStyle": {"color": color},
            "x": pos["x"],  # Manually assign x position
            "y": pos["y"],  # Manually assign y position
        }
        nodes.append(node_entry)

     # Add dummy nodes for the legend
    legend_nodes = [
        {"name": entry["name"], "symbolSize": 20, "itemStyle": {"color": entry["color"]}, "x": 0, "y": 200 + i * 50}
        for i, entry in enumerate(legend_entries)
    ]

    # Add edges for the legend (dummy edges for display)
    legend_edges = [
        {"source": entry["name"], "target": entry["name"], "lineStyle": {"color": entry["color"]}}
        for entry in legend_entries
    ]

    # Combine legend and graph nodes/edges
    nodes += legend_nodes
    edges += legend_edges
    # Create the graph visualization
    graph = (
        Graph()
        .add(
            series_name="",
            nodes=nodes,
            links=edges,
            layout="none",  # Use manual layout
            label_opts=opts.LabelOpts(is_show=True, position="right"),  # Show labels
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title=title),
            tooltip_opts=opts.TooltipOpts(formatter="{b}: €{c}"),
        )
    )

    # Render the graph to an HTML file
    output_file = "pyecharts_player_network_custom.html"
    graph.render(output_file)
    print(f"Graph visualization saved to {output_file}")

In [121]:
# define a function called draw_graph to draw the network graph with pyecharts
def draw_graph(G):
    def _get_value(node):
        value = node[1].get("player_value", 0)
        if value is None or np.isnan(value):
            return 0
        else:
            return value/1000000
    nodes = [{"name": node[0],
              "symbolSize": _get_value(node),
              "value": _get_value(node)*1000000,
              "category": 0} 
              for node in G.nodes(data=True)
              if _get_value(node) > 1]
    edges = [{"source":u, "target":v, "value":d["weight"]} for u,v,d in G.edges(data=True)]
    categories = [{"name": "Player"}]
    graph = (
        Graph()
        .add(
            "",
            nodes,
            edges,
            categories,
            repulsion=800,
            linestyle_opts=opts.LineStyleOpts(curve=0.2),
            label_opts=opts.LabelOpts(is_show=True),
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Network Graph"),
            legend_opts=opts.LegendOpts(is_show=False),
        )
    )
    output_filepath = f"./output/G_network.html"
    graph.render(output_filepath)



In [122]:
def graph_player(player, relation="Club Teammate", limit = -1):
    G = build_network(player, relation, limit)
    #draw_graph(G)
    illustrate_graph(G, player)
    # pos = nx.spring_layout(G)
    # plt.figure(figsize=(20, 20))
    # nx.draw(G, pos, node_size=node_sizes, node_color="blue", font_size= 10, with_labels=True)
    gc.collect()

In [159]:
graph_player(["Lionel Messi", "Cristiano Ronaldo"], relation="all", limit=15)

Graph visualization saved to pyecharts_player_network_custom.html


,player1,player1_name,tie,player2,player2_name,relation,player_id_x,player1_value,player_id_y,player2_value
893681,28003,Lionel Messi,Argentina,30321,Oscar Trejo,Same Country,28003,180000000.0,30321,4000000.0
893682,28003,Lionel Messi,Argentina,30654,Adrián Ricchiuti,Same Country,28003,180000000.0,30654,850000.0
893683,28003,Lionel Messi,Argentina,30677,Gustavo Cabral,Same Country,28003,180000000.0,30677,6000000.0
893684,28003,Lionel Messi,Argentina,30687,Maxi Moralez,Same Country,28003,180000000.0,30687,6000000.0
893685,28003,Lionel Messi,Argentina,30690,Sergio Romero,Same Country,28003,180000000.0,30690,7000000.0
...,...,...,...,...,...,...,...,...,...,...
5355612,28003,Lionel Messi,Paris Saint-Germain Football Club,903693,Senny Mayulu,Club Teammate,28003,180000000.0,903693,5000000.0
5355613,28003,Lionel Messi,Paris Saint-Germain Football Club,914562,Désiré Doué,Club Teammate,28003,180000000.0,914562,40000000.0
5355614,28003,Lionel Messi,Paris Saint-Germain Football Club,1047097,Yoram Zague,Club Teammate,28003,180000000.0,1047097,2500000.0
5355616,28003,Lionel Messi,Paris Saint-Germain Football Club,1075577,Joane Gadou,Club Teammate,28003,180000000.0,1075577,500000.0
